In [2]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

# 밴딧 

밴딧을 정의합니다. 

이 예에서는 4개의 손잡이를 가지는 밴딧을 이용하겠습니다. pullBandit 함수는 평균 값 0의 정규 분포로부터 랜덤한 숫자를 생성합니다. 밴딧의 수가 작을수록 +의 보상이 돌아올 가능성이 높습니다. 우리는 에이전트가 언제나 +의 보상을 가져올 손잡이를 선택하도록 학습하기를 원합니다. 

In [8]:
#밴딧의 손잡이 목록을 작성한다 
#현재 손잡이 4 (인덱스 #3) 가 가장 자주 양의 보상을 제공하도록 설정

bandit_arms = [0.2,0,-0.2,-2]
num_arms = len(bandit_arms)
def pullBandit(bandit):
    #랜덤값을 구한다
    result = np.random.randn(1)
    if result > bandit:
        #양의 보상을 반환한다
        return 1
    else:
        #음의 보상을 반환한다
        return -1

# 에이전트 

아래의 코드는 간단한 신경망을 구축합니다. 

이 신경망은 각각의 밴딧 손잡이에 대한 일련의 값들로 구성되어 있습니다. 각각의 값은 밴딧의 선택에 의해 반환되는 값의 추정치입니다. 우리는 정책 경사 방법을 이용하여 선택된 액션에 대한 값을 반환받은 보상 쪽으로 이동시켜 가는 식으로 에이전트를 업데이트시켜 나갑니다. 

In [5]:
tf.reset_default_graph()

#네트워크의 피드-포워드 부분을 구축한다 
weights = tf.Variable(tf.ones([num_arms]))
output = tf.nn.softmax(weights)

#학습 과정을 구축한다. 
#보상과 선택된 액션을 네트워크에 피드해 줌으로써 비용을 계산하고
#비용을 이용해 네트워크를 업데이트한다
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)

responsible_output = tf.slice(output,action_holder,[1])
loss = -(tf.log(responsible_output)*reward_holder)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
update = optimizer.minimize(loss)

# 에이전트 학습시키기 
이제 우리는 우리의 환경 내에서 액션을 취함으로써 에이전트를 학습시키고 보상을 받게 됩니다. 보상과 액션을 이용함으로써 우리는 시간의 경과에 따라 최고의 보상을 받게 될 액션을 보다 자주 선택하기 위해 네트워크를 어떻게 업데이트시켜 나갈 지에 대해 알 수 있게 됩니다. 

In [6]:
total_episodes = 1000 #에이전트를 학습시킬 전체 에피소드 횟수 설정
total_reward = np.zeros(num_arms) #밴딧의 손잡이에 대한 보상을 0으로 설정

init = tf.global_variables_initializer()

#텐서플로우 그래프를 론칭한다
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        
        #볼츠만 분포에 따라 액션 선택
        actions = sess.run(output)
        a = np.random.choice(actions,p=actions)
        action = np.argmax(actions == a)

        reward = pullBandit(bandit_arms[action]) #밴딧 손잡이 중 하나를 선택함으로써 보상을 받는다
        
        #네트워크를 업데이트한다
        _,resp,ww = sess.run([update,responsible_output,weights], feed_dict={reward_holder:[reward],action_holder:[action]})
        
        #보상의 총계 업데이트
        total_reward[action] += reward
        if i % 50 == 0:
            print("Running reward for the " + str(num_arms) + " arms of the bandit: " + str(total_reward))
        i+=1
print("\nThe agent thinks arm " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandit_arms)):
    print("...and it was right!")
else:
    print("...and it was wrong!")

Running reward for the 4 arms of the bandit: [ 1.  0.  0.  0.]
Running reward for the 4 arms of the bandit: [-3. -9.  8.  9.]
Running reward for the 4 arms of the bandit: [  1.  -6.  14.  28.]
Running reward for the 4 arms of the bandit: [ -4.  -4.  18.  45.]
Running reward for the 4 arms of the bandit: [ -6.  -2.  22.  59.]
Running reward for the 4 arms of the bandit: [-12.  -6.  26.  71.]
Running reward for the 4 arms of the bandit: [-13.  -3.  22.  89.]
Running reward for the 4 arms of the bandit: [ -15.   -8.   26.  100.]
Running reward for the 4 arms of the bandit: [ -14.   -9.   31.  117.]
Running reward for the 4 arms of the bandit: [  -9.   -7.   38.  133.]
Running reward for the 4 arms of the bandit: [ -17.   -4.   36.  144.]
Running reward for the 4 arms of the bandit: [ -24.   -2.   34.  159.]
Running reward for the 4 arms of the bandit: [ -25.    5.   39.  170.]
Running reward for the 4 arms of the bandit: [ -26.    4.   38.  191.]
Running reward for the 4 arms of the bandi